<a href="https://colab.research.google.com/github/Mattlee10/zone2/blob/main/25_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

연구 목적: 운동 강도에 따른 심박수 반응은 개인의 체력 및 심폐 건강을 반영한다.

기존의 생리학 모델은 실험실 환경에서 제한된 데이터로 개발되었으며, 웨어러블 기기에서 수집된 노이즈가 있는 데이터에는 적용이 어려웠다.

본 연구는 웨어러블 데이터를 활용하여 개인화된 심박수 반응을 예측하는 확장 가능한 알고리즘을 개발하고자 한다.